In [ ]:
import urllib
import csv
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import concat
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, LightSource
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn import datasets, linear_model
from mintpy.utils import readfile
import scipy.io as sio
import datetime
from dateutil.relativedelta import relativedelta
import requests
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timezone
%load_ext jupyter_ai

In [ ]:
def get_earthquakes(start_time, end_time, dem_extent, depth_range="0 10", mag_range="0 10"):
    # Define the API endpoint and parameters
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    
    depth_max, depth_min = map(lambda x: -float(x), depth_range.split())
    
    params = {
       "format": "geojson",
       "starttime": start_time,
       "endtime": end_time,
       "minlatitude": dem_extent[2],
       "maxlatitude": dem_extent[3],
       "minlongitude": dem_extent[0],
       "maxlongitude": dem_extent[1],
       "mindepth": depth_min,   # Minimum depth (in kilometers)
       "maxdepth": depth_max,   # Maximum depth (in kilometers)
    }
    
    # Make a request to the USGS API
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extract earthquake information from the API response
    earthquakes = data["features"]
    earthquake_data = []

    # Calculate the Unix timestamp in milliseconds of start and end time
    min_time = int(datetime.strptime(start_time, "%Y-%m-%d").timestamp() * 1000)
    max_time = int(datetime.strptime(end_time, "%Y-%m-%d").timestamp() * 1000)

    for quake in earthquakes:
        magnitude = quake["properties"]["mag"]
        latitude = quake["geometry"]["coordinates"][1]
        longitude = quake["geometry"]["coordinates"][0]
        depth = quake["geometry"]["coordinates"][2]
        time = quake["properties"]["time"]
        
        earthquake_data.append([time,latitude, longitude, depth, magnitude])
    
    # Create a DataFrame from the earthquake data
    columns = ["Time", "Latitude", "Longitude", "Depth", "Magnitude"]
    events_df = pd.DataFrame(earthquake_data, columns=columns)
    return events_df

def normalize_earthquake_times(events_df, start_time, end_time):
# Normalize times for colormap (use the Unix timestamp in milliseconds)
    min_time = int(datetime.strptime(start_time, "%Y-%m-%d").replace(tzinfo=timezone.utc).timestamp() * 1000)
    max_time = int(datetime.strptime(end_time, "%Y-%m-%d").replace(tzinfo=timezone.utc).timestamp() * 1000)
    norm_times = [(time - min_time) / (max_time - min_time) for time in events_df["Time"]]
    return norm_times

def modify_colormap(cmap_name = "plasma_r", exclude_beginning = 0.15, exclude_end = 0.25, show = False):
    """ modify a colormap by excluding percentages at the beginning and end """

    cmap = plt.cm.get_cmap(cmap_name)
    cmap

    num_colors_to_exclude_beginning = int(len(cmap.colors) * exclude_beginning)
    num_colors_to_exclude_end = int(len(cmap.colors) * exclude_end)
    
    # Create a custom colormap by excluding the specified percentages of colors
    colors = cmap.colors[num_colors_to_exclude_beginning:-num_colors_to_exclude_end]
    cmap_custom = LinearSegmentedColormap.from_list('cmap_custom', colors, N=256)
    
    if show is True: 
        # Create a plot with the custom colormap
        data = [[0, 1, 2, 4], [1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6]]
        plt.imshow(data, cmap=cmap_custom)
        plt.colorbar()
        plt.show()

    return cmap_custom

def add_colorbar(cmap, start_time="", end_time=""):
    # Convert date strings to datetime objects
    start_time_date = datetime.strptime(start_time, "%Y-%m-%d")
    end_time_date = datetime.strptime(end_time, "%Y-%m-%d")
    
    # Create a separate colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
    sm._A = []  # Hack to avoid normalization
    cbar = plt.colorbar(sm, ax=ax, shrink=0.5)  # Adjust the shrink value as needed
    
    # Set custom tick locations and labels on the colorbar
    ticks = np.linspace(0, 1, 5)  # You can adjust the number of ticks as needed
    tick_labels = [start_time_date + (end_time_date - start_time_date) * t for t in ticks]
    cbar.set_ticks(ticks)
    cbar.set_ticklabels([label.strftime("%Y-%m-%d") for label in tick_labels])    
    cbar.set_label("Time")

In [ ]:
def get_ticks(extent, step_size=0.2):
    # Calculate the start values for lat and long and generate sequences with step size of 0.2
    lat_start = np.ceil(extent[2] / step_size) * step_size
    lon_start = np.ceil(extent[0] / step_size) * step_size
    lats = np.arange(lat_start, extent[3], step_size)
    lons = np.arange(lon_start, extent[1], step_size)
    return lons, lats

def get_step_size(plot_extent):
    # calculate step size so that there are around 5 longitude labels
    
    step_size = (plot_extent[1]-plot_extent[0]) / 5
    if step_size <= 0.15:
        rounded_step_size = 0.1
    elif step_size <= 0.35:
        rounded_step_size = 0.2
    else:
        rounded_step_size = 0.5
    return rounded_step_size

def get_dem_extent(atr_dem):
    # get the extent which is required for plotting
    dem_extent = [float(atr_dem['X_FIRST']), float(atr_dem['X_FIRST']) + int(atr_dem['WIDTH'])*float(atr_dem['X_STEP']), 
        float(atr_dem['Y_FIRST']) + int(atr_dem['FILE_LENGTH'])*float(atr_dem['Y_STEP']), float(atr_dem['Y_FIRST'])] 
    return(dem_extent)     

def get_basemap(dem_file):
    dem, atr_dem = readfile.read(dem_file)
    dem_extent = get_dem_extent(atr_dem)
    ls = LightSource(azdeg=315, altdeg=45)
    dem_shade = ls.shade(dem, vert_exag=1.0, cmap=plt.cm.gray, vmin=-20000, vmax=np.nanmax(dem)+2500)
    return dem_shade,dem_extent

def plot_background(dem_file, plot_box = []):
    
    factory_default_figsize = plt.rcParamsDefault['figure.figsize']
    current_default_figsize = plt.rcParams['figure.figsize']
    increase_factor = 1.5
    if current_default_figsize == factory_default_figsize:
        new_default_figsize = [size * increase_factor for size in current_default_figsize]
        plt.rcParams['figure.figsize'] = new_default_figsize

    #plot DEM as background
    dem_shade, dem_extent = get_basemap(dem_file);
           
    #fig = plt.figure(figsize=(8, 8))
    fig = plt.figure()
   
    #fig, ax = plt.subplots()
    plt.rcParams['font.size'] = 12

    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.imshow(dem_shade, origin='upper', cmap=plt.cm.gray, extent=dem_extent)
    # ax.add_feature(cfeature.COASTLINE)
    
    if len(plot_box) == 0:
       plot_extent = dem_extent
    else:
       plot_extent = [plot_box[2], plot_box[3], plot_box[0], plot_box[1]]
     
    # Add latitude and longitude labels to the plot
    step_size = get_step_size(plot_extent)
    lons, lats = get_ticks(plot_extent, step_size = step_size)
    ax.set_xticks(lons, crs=ccrs.PlateCarree())
    ax.set_yticks(lats, crs=ccrs.PlateCarree())
    ax.xaxis.set_label_coords(0.5, -0.1)
    ax.yaxis.set_label_coords(-0.1, 0.5)
    
    ax.set_extent(plot_extent)

    return ax

In [ ]:
dem_file='data/demGeo.h5'
lines=sio.loadmat('./data/hawaii_lines_new.mat',squeeze_me=True);

start_time, end_time ="2022-11-29","2022-11-30"
start_time, end_time ="2022-11-30","2022-12-03"
start_time, end_time ="2022-12-03","2022-12-12"
start_time, end_time ="2022-11-28","2022-11-29"

depth_range="0 10"

plot_box=[]
plot_box=[ 19.27, 19.6, -155.79, -155.41]
plot_box=[ 19.40, 19.53, -155.68, -155.50]

cmap_name = "plasma_r"; exclude_beginning = 0.2; exclude_end = 0.2

# get shaded relief and earthquake data (normalize times for plotting)
dem_shade, dem_extent = get_basemap(dem_file)
events_df = get_earthquakes(start_time, end_time, dem_extent)
norm_times = normalize_earthquake_times(events_df, start_time, end_time)

cmap = modify_colormap( cmap_name = cmap_name, exclude_beginning = exclude_beginning, exclude_end = exclude_end, show = False)
#plasma_r = plt.cm.plasma_r

# plot background, lines and events
ax = plot_background(dem_file, plot_box = plot_box)
ax.plot(lines['Lllh'][:,0],lines['Lllh'][:,1],color='black', linestyle='dashed',linewidth=2);
ax.scatter(events_df["Longitude"],events_df["Latitude"],s=4*events_df["Magnitude"] ** 3,
         c=norm_times,cmap=cmap,alpha=0.8,transform=ccrs.PlateCarree())

add_colorbar(cmap = cmap, start_time = start_time, end_time = end_time)
ax.set_title('Events:  ' + start_time + ' - ' + end_time);